In [16]:
using Graphs
using OpenStreetMapX
using CSV
using DataFrames
using Plots
using Graphs
include("voronoi_graph.jl")
using .voronoi_graph

In [17]:
#bounds = voronoi_graph.Bounds(-3000.0, 1000.0, -1000.0, 2000.0)
#start = voronoi_graph.generate_start_sample(13, bounds)
#vorn_diagram = voronoi_graph.generate_rect_vorn_diagram(start, bounds)
#gs, ndes, edge_lst, polys = voronoi_graph.extract_graph(vorn_diagram)
#nodes_level_1 = ndes
#edges_level_1 = edge_lst

In [18]:
#!/usr/bin/env julia
"""
Convert nodes (Vector of Tuples) and edges (Vector of Tuples) to the basic JSON format.

Usage:
    include("convert_to_basic.jl")
    json_str = convert_to_basic_format(ndes, edge_lst)
    # or save to file:
    save_basic_format("output.json", ndes, edge_lst)
"""

using JSON

"""
Convert nodes and edges vectors to the basic format dictionary.

Args:
    nodes: Vector{Tuple{Float64, Float64}} - list of (x, y) coordinates
    edges: Vector{Tuple{Int64, Int64}} - list of (node1_idx, node2_idx) pairs

Returns:
    Dict with "level1" containing [[positions...], [edges...]]
"""
function convert_to_basic_format(nodes::Vector{Tuple{Float64, Float64}}, 
                                  edges::Vector{Tuple{Int64, Int64}})
    # Convert nodes to list of [x, y] arrays
    positions = [[n[1], n[2]] for n in nodes]
    
    # Convert edges to list of [n1, n2] arrays
    edge_list = [[e[1], e[2]] for e in edges]
    
    return Dict("level1" => [positions, edge_list])
end

# Alternative version for different tuple types
function convert_to_basic_format(nodes::Vector, edges::Vector)
    positions = [[Float64(n[1]), Float64(n[2])] for n in nodes]
    edge_list = [[Int(e[1]), Int(e[2])] for e in edges]
    
    return Dict("level1" => [positions, edge_list])
end

"""
Save nodes and edges to a JSON file in basic format.
"""
function save_basic_format(filepath::String, nodes, edges)
    data = convert_to_basic_format(nodes, edges)
    
    open(filepath, "w") do f
        JSON.print(f, data, 2)
    end
    
    println("Saved to $filepath")
    println("  Nodes: $(length(nodes))")
    println("  Edges: $(length(edges))")
    
    return data
end

"""
Convert and return as JSON string.
"""
function to_json_string(nodes, edges)
    data = convert_to_basic_format(nodes, edges)
    return JSON.json(data, 2)
end

to_json_string

In [19]:
save_basic_format("graph_voronoi.json", ndes, edge_lst)

Saved to graph_voronoi.json
  Nodes: 28
  Edges: 40


Dict{String, Vector{Vector{Vector{Float64}}}} with 1 entry:
  "level1" => [[[-1414.0, 1053.0], [-788.0, 1644.0], [-866.0, 2000.0], [-1585.0…

In [26]:
for i in 1:100
    bounds = voronoi_graph.Bounds(-3000.0, 1000.0, -1000.0, 2000.0)
    start = voronoi_graph.generate_start_sample(13, bounds)
    vorn_diagram = voronoi_graph.generate_rect_vorn_diagram(start, bounds)
    gs, ndes, edge_lst, polys = voronoi_graph.extract_graph(vorn_diagram)
    nodes_level_1 = ndes
    edges_level_1 = edge_lst
    
    function edge_lengths(nodes, edges)
        euclid(a, b) = hypot(a[1] - b[1], a[2] - b[2])
        return [euclid(nodes[u], nodes[v]) for (u, v) in edges]
    end
    
    edge_lengths_1 = edge_lengths(nodes_level_1, edges_level_1)
    df = DataFrame(length = edge_lengths_1)
    CSV.write("/home/adamkas/Julia/synthetic_cities/random_graphs/random_roads_$i.csv", df)
end

In [29]:
using CSV, DataFrames, Statistics

function normalize_by_mean(x::Vector{Float64})
    m = mean(x)
    m == 0.0 && error("Mean is zero; cannot normalize by mean")
    return x ./ m
end

function wasserstein1_quantile(a::Vector{Float64}, b::Vector{Float64}; grid_size::Int=2001)
    a_norm = normalize_by_mean(a)
    b_norm = normalize_by_mean(b)
    
    a_sorted = sort(a_norm)
    b_sorted = sort(b_norm)
    
    p = range(0.0, 1.0, length=grid_size)
    
    qa = quantile(a_sorted, p)
    qb = quantile(b_sorted, p)
    
    return mean(abs.(qa .- qb))
end

# Load the reference (real) data once
rl = CSV.read("/home/adamkas/Julia/synthetic_cities/ochota_road_lengths.csv", DataFrame)

# Loop over all synthetic files
#for i in 1:30
#    filepath = "/home/adamkas/Julia/synthetic_cities/random_graphs/random_roads_$i.csv"
    
#    if isfile(filepath)
#        synt = CSV.read(filepath, DataFrame)
#        dist = wasserstein1_quantile(real.length, synt.length)
#        println("File $i: Wasserstein distance = $dist")
#    else
#        println("File $i: not found, skipping")
#    end
#end

Row,length
,Float64
1,640.272
2,516.917
3,2118.07
4,551.515
5,179.022
6,1345.59
7,1267.52
8,240.819
9,1924.67


In [30]:
results = DataFrame(file_num = Int[], wasserstein = Float64[])

for i in 1:10
    filepath = "/home/adamkas/Julia/synthetic_cities/claude_rslts/claude_length_$i.csv"
    
    if isfile(filepath)
        synt = CSV.read(filepath, DataFrame)
        dist = wasserstein1_quantile(rl.length, synt.length)
        push!(results, (i, dist))
    end
end

println(results)
println("Mean Wasserstein: ", mean(results.wasserstein))
println("Std Wasserstein: ", std(results.wasserstein))

10×2 DataFrame
 Row │ file_num  wasserstein 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1   0.0583428
   2 │        2   0.0424513
   3 │        3   0.0498343
   4 │        4   0.0409265
   5 │        5   0.0188017
   6 │        6   0.00196709
   7 │        7   0.0442907
   8 │        8   0.0585023
   9 │        9   0.038486
  10 │       10   0.0539877
Mean Wasserstein: 0.04075902753298187
Std Wasserstein: 0.017915978903580085
